# Custom Functions and Modules

In [ ]:
import numpy as np
from numpy import (
    asarray,
    corrcoef,
    diag,
    eye,
    hstack,
    logical_and,
    logical_not,
    r_,
    sort,
    unique,
    zeros,
)
from numpy.linalg import inv
from numpy.random import chisquare, standard_normal

## Exercise 1


In [ ]:
def ascategory(x):
    """ """
    x = asarray(x)
    t = x.shape[0]
    ux = unique(x)
    k = ux.shape[0]
    categories = np.zeros((t, k))
    for i in range(k):
        loc = np.squeeze(x == ux[i])
        categories[loc, i] = 1.0
    return categories

In [ ]:
ascategory(["a", "b", "a", "c", "a", "b"])

## Exercise 2


In [ ]:
def gls(x, y, omega=None):
    """ """
    t, k = x.shape
    if omega is None:
        omega = eye(t)
    omega_inv = inv(omega)
    xpx = x.T @ omega_inv @ x
    xpy = x.T @ omega_inv @ y
    beta_gls = asarray(inv(xpx) @ xpy)
    return beta_gls

In [ ]:
x = standard_normal((100, 3))
y = standard_normal((100, 1))
gls(x, y)

In [ ]:
omega = np.diag(chisquare(5, size=100))
omega

In [ ]:
gls(x, y, omega)

## Exercise 3


In [ ]:
def partial_corr(x, y=None, quantile=0.5, tail="Lower"):
    """ """
    if y is not None:
        X = x.view()
        Y = y.view()
        T = X.shape[0]
        X.shape = T, 1
        Y.shape = T, 1
        z = hstack((X, Y))
    else:
        z = x
    T, K = z.shape
    corr = eye(K)
    count = zeros((K, K))
    ind = zeros((T, K), dtype=np.bool)
    for i in range(K):
        temp = sort(z[:, i].ravel())
        cutoff = int(round(quantile * T))
        threshold = temp[cutoff]
        ind[:, i] = z[:, i] < threshold
        if tail == "Upper":
            ind[:, i] = logical_not(ind[:, i])
    for i in range(K):
        for j in range(i + 1, K):
            pl = logical_and(ind[:, i], ind[:, j])
            count[i, j] = sum(pl)
            count[j, i] = count[i, j]
            if sum(pl) > 1:
                w = z[pl, :]
                w = w[:, r_[i, j]]
                corr[i, j] = corrcoef(w.T)[0, 1]
                corr[j, i] = corr[i, j]
            else:
                corr[i, j] = np.nan
                corr[j, i] = np.nan
    return corr, count